In [233]:
import numpy as np

import pandas as pd

import dask.dataframe as dd

import dask.array as da

import dask.bag as db

In [255]:
d = {'col1': [0,5,10,15], 'col2': [1,6,11,16], 'col3':[2,7,12,-1]}

df = pd.DataFrame(data=d)

df = dd.from_pandas(df, npartitions=2)

df

,col1,col2,col3
npartitions=2,,,
0,int64,int64,int64
2,...,...,...
3,...,...,...


In [256]:

df["image_guide"] = np.nan
df.compute()

,col1,col2,col3,image_guide
0,0,1,2,NaN
1,5,6,7,NaN
2,10,11,12,NaN
3,15,16,-1,NaN


In [259]:
df.set_index("col3")
df.repartition(npartitions=2)
df.compute()

,col1,col2,col3,image_guide
0,0,1,2,NaN
1,5,6,7,NaN
2,10,11,12,NaN
3,15,16,-1,NaN


In [29]:
df = df.sample(frac=1).reset_index(drop=True)
df.compute()

,col1,col2,image_guide
0,1,3,NaN
0,2,4,NaN


In [30]:
data_frame = df[["col1","col2"]]
data_frame.compute()

,col1,col2
0,1,3
0,2,4


In [31]:
meta_df = pd.DataFrame(columns=["x", "y","z"], dtype=int)
meta_df

,x,y,z


In [32]:
def speaker_emb_preprocess(x):
    return x*x*x

data_frame['col3'] = data_frame['col1'].apply(lambda x: speaker_emb_preprocess(x),meta=('1', 'object'))
data_frame

,col1,col2,col3
npartitions=2,,,
,int64,int64,object
,...,...,...
,...,...,...


In [33]:
meta_df = pd.DataFrame([[1,2,4],[1,4,5]], dtype=int)
type("meta_df")

str

In [34]:
type(np.zeros((1,768)).tolist())

list

In [35]:
data_frame = data_frame.drop(['col1'], axis=1)
data_frame

,col2,col3
npartitions=2,,
,int64,object
,...,...
,...,...


In [36]:
data_frame[['col2','col3']]

,col2,col3
npartitions=2,,
,int64,object
,...,...
,...,...


In [37]:
np.array(data_frame["col2"])

array([3, 4])

In [38]:
for index, row in data_frame.iterrows():
    print(row['col2'])

3
4


In [39]:
data_frame.loc[1,"col2"]=2

TypeError: '_LocIndexer' object does not support item assignment

In [ ]:
from matplotlib import pyplot as plt
x = []
x.append(0)
x.append(1)
x.append(2)
x

In [ ]:
fig = plt.figure()
plt.plot(range(10))

plt.title("Training Loss")
plt.xlabel("Training Sample (x500)")
plt.ylabel("MSE Loss")
fig.savefig("save_file_name.png")
plt.close()

In [ ]:
import random
# For a single Element
random.choices([True, False], weights=[100, 0])[0]

In [ ]:
from PIL import Image
import numpy as np


image_size = 128

def make_square(im, min_size=128, fill_color=(0, 0, 0, 0)):
    x, y = im.size
    size = max(min_size, x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    im.close()
    return new_im

def image_guide_preprocess_low_res_dummy(path):
    #print(face_id)

    image_guide = np.zeros((1, 768))
    return image_guide

def image_guide_preprocess_low_res(path):
    #print(face_id)


    if(False): #never
        image_guide = np.zeros((49, 768))
        #print(image_guide.shape)
    else:

        image = Image.open(path).convert('RGB')

        #print(image.size)

        w_s = image_size / (1+2 * 0.4)
        h_s = image_size / (1+2 * 0.4)

        image = image.crop((0.2*w_s, 0.0*h_s, 1.6*w_s, 1.4*h_s))
        image = image.resize((128,128))
        print(image.size)
        #image = im

        #image = image.resize((16,16))
        im = image
        

        print('saving')
        image.save('opop.png')

        #print(np.array(image,np.float32).shape)

        pix = np.array(image, np.float32)
        pix = np.moveaxis(pix, -1, 0)

        pix = pix / 255
        image.close()
        im.close()
        return pix.tolist()
    return image_guide


In [ ]:
im = '/media/gamal/Passport/Datasets/VoxCeleb2Test/Voxceleb2TestFaces/mp4/id01106/45TAeIEqhcU/00001_face_1.png'

In [ ]:
print(im)

In [ ]:
image_guide_preprocess_low_res(im)

In [ ]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [51]:
from itertools import islice
import csv
import sys
import os
import glob
csv.field_size_limit(sys.maxsize)
class CsvDataset(Dataset):

    def __init__(self, csv_folder):

        self.csv_folder = csv_folder

    def __len__(self):
        _, _, files = next(os.walk(self.csv_folder))
        file_count = len(files)
        print('found ' + str(file_count) + ' files.')
        return file_count

    def __getitem__(self, index):
        file = sorted(glob.glob(self.csv_folder + '*.csv'))[index]
        df = pd.read_csv(file)
        print(df.head(1))
        df['INPUT'] = np.fromstring(s[1:-1], dtype=np.int, sep=' ')
        #df['INPUT'] = df['INPUT'].astype('numpy.ndarray')
        print(df['INPUT'][0])
        torch_tensor = torch.tensor(df.head(1).values)
        return torch_tensor

In [52]:
face_dataset = CsvDataset(csv_folder='train/')

In [53]:
train_loader = torch.utils.data.DataLoader(face_dataset,
        batch_size=1)

In [54]:
for x in train_loader:
    print(x)
    break

found 1 files.
   Unnamed: 0                                              INPUT  \
0           0  [[0. 0. 0. ... 0. 0. 1.]\n [0. 0. 0. ... 0. 0....   

                                          image_path  \
0  [[[0.23921569 0.23921569 0.23921569 ... 0.3019...   

                                       low_res_image  
0  [[[0.24313726 0.6313726  0.6784314  0.7529412 ...  


TypeError: data type 'numpy.ndarray' not understood

In [70]:
names = np.arange(10)
charar = np.chararray((1, 10))
charar[:] = 'i'
names = charar.astype('str') + names.astype('str')
meta_df = pd.DataFrame(columns=names.astype(str), dtype=int)
meta_df

,"(i0, i1, i2, i3, i4, i5, i6, i7, i8, i9)"


array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U21')